In [5]:
pip install matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 1.5 MB/s eta 0:00:00a 0:00:01m

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

mine_df = pd.read_csv('Mine.csv')

print("===Mine.csv Dataset===")
mine_df.info()
print("\n===First 5 Rows of Mine.csv===")
print(mine_df.head())

sales_df = pd.read_csv('Sales.csv')
print("\n===Sales.csv Dataset===")
sales_df.info()
print("\n===First 5 Rows of Sales.csv===")
print(sales_df.head())

===Mine.csv Dataset===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Duration  31 non-null     int64  
 1   Date      29 non-null     object 
 2   Pulse     30 non-null     float64
 3   Maxpulse  29 non-null     float64
 4   Calories  27 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 1.3+ KB

===First 5 Rows of Mine.csv===
   Duration         Date  Pulse  Maxpulse  Calories
0        60  2023/10/01'  110.0     130.0     409.1
1        60  2023/10/02'  117.0     145.0     479.0
2        60  2023/10/03'  103.0     135.0     340.3
3        45  2023/10/04'  109.0     175.0     282.4
4        45  2023/10/05'  117.0     150.0     405.1

===Sales.csv Dataset===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         -------

In [14]:
# check for missing values

print("\n===Missing Values in Mine.csv===")
print(mine_df.isna().sum())

# duplicate rows
print("\n===Duplicate Rows in Mine.csv===")
print(mine_df.duplicated().sum())

# Handle date format issues 
mine_df['Date'] = mine_df['Date'].astype(str)
mine_df['Date'] = mine_df['Date'].str.replace("'", "")

# Convert Date to datetime format
mine_df['Date'] = pd.to_datetime(mine_df['Date'], errors='coerce')
# check for missing values in Mine.csv
mine_df['Calories'] = mine_df['Calories'].fillna(mine_df['Calories'].median())
mine_df['Maxpulse'] = mine_df['Maxpulse'].fillna(mine_df['Maxpulse'].median())
mine_df['Pulse'] = mine_df['Pulse'].fillna(mine_df['Pulse'].median())

# # check for outliers in duration
# print("\n===Outliers in Duration===")
# print(mine_df['Duration'].describe())

# Let's identify outliers using IQR method
Q1 = mine_df['Duration'].quantile(0.25)
Q3 = mine_df['Duration'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"\nOutliers in Duration (values outside {lower_bound}-{upper_bound}):")
outliers = mine_df[(mine_df['Duration'] < lower_bound) | (mine_df['Duration'] > upper_bound)]
print(outliers[['Duration', 'Date']])

# Handle outliers (e.g., cap them or remove them)
mine_df_cleaned = mine_df.copy()
mine_df_cleaned.loc[mine_df_cleaned['Duration'] > upper_bound, 'Duration'] = upper_bound

print("\nCleaned Mine.csv data:")
print(mine_df_cleaned.head())
print(mine_df_cleaned.info())



===Missing Values in Mine.csv===
Duration    0
Date        3
Pulse       0
Maxpulse    0
Calories    0
dtype: int64

===Duplicate Rows in Mine.csv===
0

Outliers in Duration (values outside 60.0-60.0):
    Duration       Date
3         45 2023-10-04
4         45 2023-10-05
7        400 2023-10-08
9         30 2023-10-10
17        45        NaT

Cleaned Mine.csv data:
   Duration       Date  Pulse  Maxpulse  Calories
0        60 2023-10-01  110.0     130.0     409.1
1        60 2023-10-02  117.0     145.0     479.0
2        60 2023-10-03  103.0     135.0     340.3
3        45 2023-10-04  109.0     175.0     282.4
4        45 2023-10-05  117.0     150.0     405.1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Duration  31 non-null     int64         
 1   Date      28 non-null     datetime64[ns]
 2   Pulse     31 non-null     float64     

In [15]:
# Check for missing values
print("\nMissing values in Sales.csv:")
print(sales_df.isna().sum())

# Check for duplicate rows
print("\nDuplicate rows in Sales.csv:", sales_df.duplicated().sum())

# Handle negative quantities
print("\nRows with negative quantities:")
print(sales_df[sales_df['Quantity'] < 0])


sales_df_cleaned = sales_df.copy()
sales_df_cleaned = sales_df_cleaned[sales_df_cleaned['Quantity'] >= 0]

# Clean customer names - remove empty values
sales_df_cleaned = sales_df_cleaned[sales_df_cleaned['Customer Name'].notna()]

# Standardize date format
sales_df_cleaned['Order Date'] = sales_df_cleaned['Order Date'].str.replace("'", "")
sales_df_cleaned['Order Date'] = pd.to_datetime(sales_df_cleaned['Order Date'], errors='coerce')

# Fill missing values in Unit Price
sales_df_cleaned['Unit Price'] = sales_df_cleaned['Unit Price'].fillna(sales_df_cleaned.groupby('Product')['Unit Price'].transform('mean'))

#  Recalculate Total Revenue where it's NaN or incorrect
sales_df_cleaned['Calculated Revenue'] = sales_df_cleaned['Quantity'] * sales_df_cleaned['Unit Price']
mask = sales_df_cleaned['Total Revenue'].isna() | (abs(sales_df_cleaned['Total Revenue'] - sales_df_cleaned['Calculated Revenue']) > 1)
sales_df_cleaned.loc[mask, 'Total Revenue'] = sales_df_cleaned.loc[mask, 'Calculated Revenue']

# Check the cleaned data
print("\nCleaned Sales.csv data:")
print(sales_df_cleaned.head())
print(sales_df_cleaned.info())


Missing values in Sales.csv:
Order ID         0
Customer Name    1
Order Date       0
Product          0
Quantity         1
Unit Price       1
Total Revenue    1
dtype: int64

Duplicate rows in Sales.csv: 1

Rows with negative quantities:
   Order ID Customer Name  Order Date   Product  Quantity  Unit Price  \
7      1007    Jane Smith  07/01/2024  Widget C      -6.0        70.0   

   Total Revenue  
7         -420.0  

Cleaned Sales.csv data:
   Order ID  Customer Name Order Date   Product  Quantity  Unit Price  \
0      1001       John Doe 2024-01-01  Widget A      10.0        25.0   
1      1002     Jane Smith 2024-01-02  Widget B       5.0        40.0   
3      1004  Alice Johnson 2024-04-01  Widget C       3.0         NaN   
4      1005      Bob Brown        NaT  Widget B      10.0        40.0   
5      1006       John Doe 2024-06-01  Widget A       4.0        25.0   

   Total Revenue  Calculated Revenue  
0          250.0               250.0  
1          200.0               20

In [16]:
# analysis for mine.csv
print("\n===Analysis of Mine.csv===")

print("\nStatistical summary for Mine.csv:")
print(mine_df.describe())

print("\nCorrelation matrix:")
correlation  = mine_df_cleaned.corr()
print(correlation)

# duration vs calories
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Duration', y='Calories', data=mine_df_cleaned)
plt.title('Relation between Duration and Calories Burned')
plt.savefig('duration_vs_calories.png')
plt.close()

# distribution of pulse
plt.figure(figsize=(10, 6))
sns.histplot(mine_df_cleaned['Pulse'], kde=True)
plt.title('Distribution of Pulse Rate')
plt.savefig('pulse_distribution.png')
plt.close()

print("\n=== Sales.csv Analysis ===")

# Sales by product
product_sales = sales_df_cleaned.groupby('Product')['Total Revenue'].sum().sort_values(ascending=False)
print("\nTotal Revenue by Product:")
print(product_sales)

# Sales by customer
customer_sales = sales_df_cleaned.groupby('Customer Name')['Total Revenue'].sum().sort_values(ascending=False)
print("\nTotal Revenue by Customer:")
print(customer_sales.head())

# sales by product visualization
plt.figure(figsize=(12, 6))
product_sales.plot(kind='bar')
plt.title('Total Revenue by Product')
plt.ylabel('Total Revenue')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('revenue_by_product.png')
plt.close()

# sales trend over time visualization
sales_over_time = sales_df_cleaned.groupby(pd.Grouper(key='Order Date', freq='D'))['Total Revenue'].sum()
plt.figure(figsize=(14, 6))
sales_over_time.plot()
plt.title('Sales Trend Over Time')
plt.ylabel('Total Revenue')
plt.grid(True)
plt.tight_layout()
plt.savefig('sales_trend.png')
plt.close()


===Analysis of Mine.csv===

Statistical summary for Mine.csv:
         Duration                           Date       Pulse    Maxpulse  \
count   31.000000                             28   31.000000   31.000000   
mean    68.548387  2023-10-15 05:08:34.285714176  118.725806  123.354839   
min     30.000000            2023-10-01 00:00:00   92.000000  100.000000   
25%     60.000000            2023-10-07 18:00:00  103.000000  107.500000   
50%     60.000000            2023-10-15 00:00:00  115.500000  125.000000   
75%     60.000000            2023-10-22 12:00:00  133.000000  132.000000   
max    400.000000            2023-10-31 00:00:00  162.000000  175.000000   
std     61.889600                            NaN   17.480388   17.081273   

         Calories  
count   31.000000  
mean   300.219355  
min    193.800000  
25%    240.800000  
50%    282.400000  
75%    345.750000  
max    479.000000  
std     68.828194  

Correlation matrix:
          Duration      Date     Pulse  Maxpulse  C